In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession

sc = SparkContext("local", "pbl-1")
spark = SparkSession(sc)

In [3]:
data = sc.textFile("facebook_combined.txt")

parse_data = data.flatMap(lambda line: line.split('\n')) \
            .map(lambda l:l.split(' ')).map(lambda l: [(int(l[0]),int(l[1])),(int(l[1]),int(l[0]))]).flatMap(lambda t: [t[0],t[1]])

In [4]:
group = parse_data.groupByKey().mapValues(list)
new_group = group.flatMap(lambda l: [(l[0],(len(l[1]), sorted(l[1])))])
#print(new_group.collect())

In [5]:
order = new_group.sortBy(lambda v: v[1])
R = order.mapValues(lambda v: v[1]).collect()
#print(R)

In [ ]:
import math
import time

r_set = [0.6, 0.7, 0.8, 0.9]
set_idx = {id:i for i,(id,x) in enumerate(R)}

for r in r_set:
    print("# when r = ",r)
    start = time.time()
    
    S = []
    I = [[] for _ in range(len(R))] # inverted list

    for (x_id, x) in R:
        max_probe_prefix = len(x) - math.ceil(r*len(x)) + 1
        max_index_prefix = len(x) - math.ceil(2*len(x)*r/(r+1)) + 1
        prefix_x = [0]
        prefix_y = [0]
        A = [0 for _ in range(len(R))] # overlap score for y

        for i in range(1,max_probe_prefix+1):
            a = math.ceil((2*len(x)-i+1)*r/(r+1))
            prefix_x.append(len(x)-a+1)
            prefix_y.append(len(x)-i+1-a+1)

        for i in range(1,max_probe_prefix+1):
            w = x[i-1]
            w_idx = set_idx[w]

            for ((y_id, y),j) in I[w_idx]:
                if len(y) < r*len(x):
                    I[w_idx].remove(((y_id,y),j))
                elif j > prefix_y[len(x)-len(y)+1]:
                    I[w_idx].remove(((y_id,y),j))
                elif i > prefix_x[len(x)-len(y)+1]: continue
                else:
                    A[set_idx[y_id]] = A[set_idx[y_id]] + 1 # 후보 확정

        for i in range(1,max_index_prefix+1): # x의 원소에 대한 inverted list 추가
            w = x[i-1]
            w_idx = set_idx[w]
            I[w_idx].append(((x_id,x),i))

        for i in range(len(R)): # 후보 검증 후 S에 추가
            (y_id, y) = R[i]

            if A[i] == 0: continue # prefix 안에 공통 토큰이 하나도 없다면 제외

            a = math.ceil((len(x)+len(y))*r/(r+1))

            (p,q,cnt) = (0,0,0)
            
            while(p<len(x) and q<len(y)):# 공통 토큰 개수 카운트
                if x[p] == y[q]:
                    cnt = cnt+1
                    p = p+1
                    q = q+1
                elif cnt+min(len(x)-p-1,len(y)-q-1) < a: break # positional filtering
                elif x[p] < y[q]: p = p+1
                else: q = q+1
            if cnt >= a and y_id not in x: # x와 y가 이미 친구인 경우는 제외
                S.append((x_id, y_id))
    
    end = time.time()
    print(S)
    
    print('duration : ', end-start)